<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Pytorch/LearningPytorch_2_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris

In [2]:
# GPU の設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Iris データセット読み込み
x, t = load_iris(return_X_y=True)

In [4]:
x = torch.tensor(x, dtype = torch.float32)
t = torch.tensor(t, dtype = torch.int64)

In [5]:
# DataSetに格納
dataset = torch.utils.data.TensorDataset(x,t)

In [6]:
# datasetの分割
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [7]:
# ランダムに分割する
torch.manual_seed(7)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [8]:
# ミニバッチ学習
batch_size = 10

In [9]:
# Dataloaderに格納、trainだけshuffle
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

In [10]:
# モデル定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.L1 = nn.Linear(4,4)
        self.L2 = nn.Linear(4,3)
    def forward(self, x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        return x

In [11]:
# 学習後の正答率の計算
def calc_acc(data_loader):
    with torch.no_grad():#勾配計算しない
        accs = []
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            # argmax index of max
            # dim (short for 'dimension') is the torch equivalent of 'axis' in NumPy.
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0
            acc = acc / len(t)
            #print(acc)
            accs.append(acc)
        avg_acc = torch.tensor(accs).mean()
        print('Accuracy: {:.1f}%'.format(avg_acc * 100))

In [14]:
# 学習モデルループ実装
max_epoch = 10
# インスタンス化
torch.manual_seed(0)
net = Net().to(device)
# 目標関数の設定
criterion = nn.CrossEntropyLoss()
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1)
print("学習開始 エポック数", max_epoch)
# 学習ループ
for epoch in range(max_epoch):
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)

        optimizer.zero_grad()
        y = net(x)
        # 正解率を出力?
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(y_label == t) * 1.0
        acc = acc / len(t)
        loss = criterion(y, t)
        print("accuracy: ", acc, "loss: ", loss.item())# item => tensor.Tensor => float
        loss.backward()
        optimizer.step()
print("\n学習結果の検証")
calc_acc(val_loader)

学習開始 エポック数 10
accuracy:  tensor(0.5000, device='cuda:0') loss:  1.0779688358306885
accuracy:  tensor(0.5000, device='cuda:0') loss:  1.0664923191070557
accuracy:  tensor(0.7000, device='cuda:0') loss:  0.9774702787399292
accuracy:  tensor(0.5000, device='cuda:0') loss:  0.9559988975524902
accuracy:  tensor(0.4000, device='cuda:0') loss:  0.9044672846794128
accuracy:  tensor(0.4000, device='cuda:0') loss:  0.9096088409423828
accuracy:  tensor(0.4000, device='cuda:0') loss:  0.8713811635971069
accuracy:  tensor(0.5000, device='cuda:0') loss:  0.7703483700752258
accuracy:  tensor(0.3000, device='cuda:0') loss:  0.8946412801742554
accuracy:  tensor(0.9000, device='cuda:0') loss:  0.6919468641281128
accuracy:  tensor(0.7000, device='cuda:0') loss:  0.6308976411819458
accuracy:  tensor(0.7000, device='cuda:0') loss:  0.7791956067085266
accuracy:  tensor(0.9000, device='cuda:0') loss:  0.6671345829963684
accuracy:  tensor(0.8000, device='cuda:0') loss:  0.8308623433113098
accuracy:  tensor(0.